In [ ]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import pdb
import numpy as np
import skimage
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from matplotlib import pyplot
# class that defines and loads the weed dataset
class WeedDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define two class
		self.add_class("dataset", 1, "crop")
		self.add_class("dataset", 2, "weed")
		# define data locations
		images_dir = dataset_dir + '/raw images/'
		annotations_dir = dataset_dir + '/annotations/'
		
		# find all images
		for filename in listdir(images_dir):
			# extract image id
			image_id = filename[:-4]
			#print('IMAGE ID: ',image_id)
			# skip bad images
			if (image_id == '.ipynb_checkpo'):
				continue
			# skip all images after 115 if we are building the train set
			if int(image_id) in [48,51,57,95]:
				continue
			if is_train and int(image_id) >= 32279:
				continue
			# skip all images before 115 if we are building the test/val set
			if not is_train and int(image_id) < 32279:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		#for box in root.findall('.//bndbox'):
		for box in root.findall('.//object'):
			name = box.find('name').text
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			#coors = [xmin, ymin, xmax, ymax, name]
			coors = [xmin, ymin, xmax, ymax, name]
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		#pdb.set_trace()
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'crop'):
				masks[row_s:row_e, col_s:col_e, i] = 2
				class_ids.append(self.class_names.index('crop'))
			else:
				masks[row_s:row_e, col_s:col_e, i] = 1
				class_ids.append(self.class_names.index('weed'))
		return masks, asarray(class_ids, dtype='int32')
	# load an image reference
	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# define a configuration for the model
class PWeedConfig(Config):
	# define the name of the configuration
	NAME = "weed_cfg"
	# number of classes (background + weed + crop)
	NUM_CLASSES = 1 + 2; GPU_COUNT = 1; IMAGES_PER_GPU = 1; IMAGE_MAX_DIM=1280; IMAGE_MIN_DIM=720
    
# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		print(image_id); image1 = dataset.load_image(image_id)
		mask, class_ids = dataset.load_mask(image_id)
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image1, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		APs.append(AP)
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP

# prepare train set
train_set = WeedDataset()
train_set.load_dataset('Ronin_OPEN_DB', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = WeedDataset()
test_set.load_dataset('Ronin_OPEN_DB', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# prepare config
config = PWeedConfig()
config.display()
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=config)
# load weights and exclude the output layers
model.load_weights('mask_rcnn_weed_cfg_0004.h5', by_name=True)
# evaluate model on training dataset
train_mAP = evaluate_model(train_set, model, config)
print("Train mAP: %.3f" % train_mAP)
# evaluate model on test dataset
test_mAP = evaluate_model(test_set, model, config)
print("Test mAP: %.3f" % test_mAP)